In [1]:
import sys, os
import sqlite3
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.2 pyspark-shell' 

In [3]:
sc     = SparkContext( appName='streamTest' )  
ssc    = StreamingContext(sc, 5)

In [4]:
def insertData(data):
    try:
        sql = 'insert into ktest values(?,?)'
        db  = sqlite3.connect(r'C:\my.db')
        db.excute( sql, data )
        db.commit( )  # insert, delete, update
        db.close()
    except Exception as err:
        print("에러:", err )

In [5]:
def recvData( data ):
    insertData( data ) 
    print("data=", data)  # ('키','값')
    return data

In [6]:
topic  = 'sample'
kvs    = KafkaUtils.createStream(ssc, 'localhost', 'raw-event-streaming-consumer', {topic:1})
# ============================================================================================
# 메시지 수신시 redvData call
lines  = kvs.map( recvData )  # 콜백 함수 등록
lines.pprint()                # 상태메시지
# ============================================================================================
ssc.start()                   # 폴링 시작 (5초에 한번씩 카프카 클러스터에서 smaple topic 확인.)
ssc.awaitTerminationOrTimeout(2)

False

In [7]:
# 폴링 중지.
sc.stop()                  